---
# **LIBRARY**
---

In [1]:
# Library sistem
import os

# Library dasar
import numpy as np
import pandas as pd

# Library visualisasi
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

# Library statistika
from statsmodels.tsa.stattools import acf, pacf

# Library evaluasi
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

---
# **CONFIGURATION**
---

In [2]:
# ============================
# Configuration
# ============================
METADATA_PATH = "../data/processed/(K3 - Transformasi per jam) Beban listrik.csv" # Pastikan File telah ditambahkan
OUTPUT_PATH = "../hasil/ssa"

# Membuat direktori jika belum ada
os.makedirs(OUTPUT_PATH, exist_ok=True)
print(f"Output directory ready: {OUTPUT_PATH}")

Output directory ready: ../hasil/ssa


---
# **LOAD METADATA**
---

In [3]:
# Memanggil data
df_siap = pd.read_csv(METADATA_PATH)
df_siap['TANGGAL_JAM'] = pd.to_datetime(df_siap['TANGGAL_JAM'])
df_siap['TAHUN'] = df_siap['TANGGAL_JAM'].dt.year

df_siap.head()

,TANGGAL_JAM,BEBAN,TAHUN
0,2016-01-01 00:00:00,5204.705,2016
1,2016-01-01 01:00:00,5208.790,2016
2,2016-01-01 02:00:00,5050.520,2016
3,2016-01-01 03:00:00,4873.250,2016
4,2016-01-01 04:00:00,4844.160,2016


---
## **1. Dekomposisi**
---

In [4]:
L = 168 # Panjang jendela 168 = (24*7)

In [5]:
SSA_DATA_PATH = f"{OUTPUT_PATH}/data"
SSA_KOMPONEN_PATH = f"{OUTPUT_PATH}/komponen"


os.makedirs(SSA_DATA_PATH, exist_ok=True)
os.makedirs(SSA_KOMPONEN_PATH, exist_ok=True)


### **Embadding**

In [6]:
def windowing(F: np.ndarray, L: int) -> np.ndarray:
    """
    Membentuk matriks trajectory SSA (windowing / embedding).

    Parameters
    ----------
    F : np.ndarray
        Deret waktu 1D, panjang N.
    L : int
        Panjang window (1 <= L <= N)

    Returns
    -------
    np.ndarray
        Matriks trajectory berukuran (L, K), dengan K = N - L + 1.
        Setiap kolom merepresentasikan vektor lag sepanjang L.
    """
    F = np.asarray(F).ravel()  # pastikan 1D
    N = len(F)
    if not (1 <= L <= N):
        raise ValueError("L harus antara 1 dan panjang data F.")
    K = N - L + 1
    X = np.column_stack([F[i:i + L] for i in range(K)])
    return X

In [14]:
X = windowing(df_siap['BEBAN'], L)
print(df_siap['BEBAN'].values)
print(X)

np.save(f'{OUTPUT_PATH}/X_{L}.npy', X)
np.savetxt(f'{OUTPUT_PATH}/X_{L}.csv', X, delimiter=',', fmt='%d')

[5204.705 5208.79  5050.52  ... 8455.14  7918.88  7510.35 ]
[[5204.705 5208.79  5050.52  ... 7640.71  7288.01  7078.5  ]
 [5208.79  5050.52  4873.25  ... 7288.01  7078.5   6934.38 ]
 [5050.52  4873.25  4844.16  ... 7078.5   6934.38  6816.52 ]
 ...
 [7347.71  6791.18  6189.06  ... 9305.29  9026.48  8455.14 ]
 [6791.18  6189.06  5877.48  ... 9026.48  8455.14  7918.88 ]
 [6189.06  5877.48  5644.61  ... 8455.14  7918.88  7510.35 ]]


### **Singular Value Decomposition (SVD)** 

In [7]:
# Fungsi dekomposisi
def dekomposisi(X, k=None):
  d = np.linalg.matrix_rank(X)
  U, Sigma, V = np.linalg.svd(X, full_matrices=False)
  V = V.T

  if k is None:
      k = d  # Gunakan semua komponen jika k tidak diberikan

  X_komponen = np.array([Sigma[i] * np.outer(U[:, i], V[:, i]) for i in range(k)])

  if not np.allclose(X, X_komponen.sum(axis=0), atol=1e-10):
    print("WARNING: The sum of X's elementary matrices is not equal to X!")

  return X_komponen, U, Sigma, V, d

# Fungsi hankelisasi
def Hankelise(X):
    """
    Hankelises the matrix X, returning H(X).
    """
    L, K = X.shape
    transpose = False
    if L > K:
        # Hanya berlaku untuk matriks berukuran L < K.
        # Untuk L > K, maka ditranspose terlebih dahulu.
        # Memberikan tanda bahwa matriks ditranspose.
        X = X.T
        L, K = K, L
        transpose = True

    HX = np.zeros((L,K))

    # Diagonal Averaging
    for m in range(L):
        for n in range(K):
            s = m+n
            if 0 <= s <= L-1:
                for l in range(0,s+1):
                    HX[m,n] += 1/(s+1)*X[l, s-l]
            elif L <= s <= K-1:
                for l in range(0,L-1):
                    HX[m,n] += 1/(L-1)*X[l, s-l]
            elif K <= s <= K+L-2:
                for l in range(s-K+1,L):
                    HX[m,n] += 1/(K+L-s-1)*X[l, s-l]
    if transpose:
        return HX.T
    else:
        return HX

def X_to_TS(X_i):
    """Averages the anti-diagonals of the given elementary matrix, X_i, and returns a time series."""
    # Membalik urutan X_i
    X_rev = X_i[::-1]
    return np.array([X_rev.diagonal(i).mean() for i in range(-X_i.shape[0]+1, X_i.shape[1])])


In [18]:
# Mendekompsisi matriks trajektori X
X_komponen, U, Sigma, V, d = dekomposisi(X)

In [22]:
print(U.shape)
print(Sigma.shape)
print(V.shape)

(168, 168)
(168,)
(77690, 168)


In [21]:
np.save(f'{SSA_DATA_PATH}/X_komponen.npy', X_komponen)
np.save(f'{SSA_DATA_PATH}/U.npy', U)
np.save(f'{SSA_DATA_PATH}/Sigma.npy', Sigma)
np.save(f'{SSA_DATA_PATH}/V.npy', V)
np.save(f'{SSA_DATA_PATH}/d.npy', d)

In [8]:
X_komponen = np.load(f'{SSA_DATA_PATH}/X_komponen.npy')
U = np.load(f'{SSA_DATA_PATH}/U.npy')
Sigma = np.load(f'{SSA_DATA_PATH}/Sigma.npy')
V = np.load(f'{SSA_DATA_PATH}/V.npy')
d = np.load(f'{SSA_DATA_PATH}/d.npy')

In [ ]:
F_i = [X_to_TS(X_komponen[i]) for i in range(L)]  # Konversi semua komponen

In [9]:
t = range(len(df_siap.index))
colors = plt.get_cmap('tab20').colors

# Simpan setiap plot sebagai PNG dan PDF terpisah
for i in range(L):
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(t, F_i[i], lw=2, color=colors[i % len(colors)])
    ax.set_title(f"Komponen {i}", fontsize=16)
    ax.set_xlabel("Waktu", fontsize=14)
    ax.set_ylabel("Nilai", fontsize=14)
    ax.tick_params(axis='both', which='major', labelsize=12)

    # Simpan gambar PNG
    png_filename = os.path.join(SSA_KOMPONEN_PATH, f"komponen_{i}.png")
    plt.savefig(png_filename, dpi=300, bbox_inches="tight")

    # Simpan gambar PDF
    pdf_filename = os.path.join(SSA_KOMPONEN_PATH, f"komponen_{i}.pdf")
    plt.savefig(pdf_filename, dpi=300, bbox_inches="tight")

    plt.close(fig)

print(f"✅ Semua {L} plot telah disimpan sebagai PNG dan PDF di folder: {SSA_KOMPONEN_PATH}")

✅ Semua 168 plot telah disimpan sebagai PNG dan PDF di folder: ../hasil/ssa/komponen


---
## **2. Rekonstruksi**
---

In [ ]:
SSA_REKONSTRUKSI_PATH = f"{OUTPUT_PATH}/rekonstruksi"

os.makedirs(SSA_REKONSTRUKSI_PATH, exist_ok=True)
